# Scraping data from Retrosheet.org: Part 2 - parsing the split page

In [112]:
from bs4 import BeautifulSoup;
import requests;
import re;
import pandas as pd;
import numpy as np;
from IPython.display import clear_output;

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; \
    Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'};
retro_page_affix = "https://www.retrosheet.org/boxesetc/";


In [19]:
df_player_career_stats = pd.read_csv("player_career_split_url.csv", usecols = ["ID", "Name", "split_url"]);
df_player_career_stats

,ID,Name,split_url
0,0,David Aardsma,https://www.retrosheet.org/boxesetc/A/Jaardd00...
1,1,Hank Aaron,https://www.retrosheet.org/boxesetc/A/Jaaroh10...
2,2,Tommie Aaron,https://www.retrosheet.org/boxesetc/A/Jaarot10...
3,3,Don Aase,https://www.retrosheet.org/boxesetc/A/Jaased00...
4,4,Andy Abad,https://www.retrosheet.org/boxesetc/A/Jabada00...
...,...,...,...
16004,19913,Bob Zupcic,https://www.retrosheet.org/boxesetc/Z/Jzupcb00...
16005,19914,Frank Zupo,https://www.retrosheet.org/boxesetc/Z/Jzupof10...
16006,19915,Paul Zuvella,https://www.retrosheet.org/boxesetc/Z/Jzuvep00...
16007,19916,George Zuverink,https://www.retrosheet.org/boxesetc/Z/Jzuveg10...


In [12]:
# Process if have chance. Have a larger chance to be blocked by Retrosheet.
df_player_yearly_stats = pd.read_csv("player_season_split_url.csv", usecols = ["ID", "Name", "Season", "Team", "split_url"]);

In [137]:
# The columns of the dataframe
columns_career = ["ID", "Name", "Situation", "G", "AB", "R", "H", "2B", "3B", "HR", "RBI", "BB", "IBB", "SO", "HBP", "SH", \
                 "SF", "XI", "ROE", "GDP", "SB", "CS", "AVG", "OBP", "SLG"]; # len() = 25
situations = ["Total", "Home", "Away", "vs RHP", "vs LHP", "Day", "Night", "None On", "Men On", "RISP", "Close & Late", \
              "Bases Loaded", "January", "February", "March", "April", "May", "June", "July", "August", "September", \
              "October", "November", "December", "1st", "2nd", "3rd", "4th", "5th", "6th", "7th", "8th", "9th", \
              "AT P ", "AT C ", "AT 1B", "AT 2B", "AT 3B", "AT SS", "AT LF", "AT CF", "AT RF", "AT OF", "AT DH", "AT PH", \
              "AT PR", "AT H", "AT >1"]; # len() = 48
situations_len = [len(x) for x in situations];
(situations_len[33], situations_len[34]) = (4,4);
# A subtlety here. In situations, there is a blank on the situations "AT C " and "AT P ".
# This is intentional to make sure it does not match "AT CF", "AT PH", "AT PR" by mistake.
situations_no_g = [3,4,7,8,9,10,11]; # situation tags with no G provided.

In [180]:
def scrape_player_split(entry):
    ''' Scrape a player in the dataframe, specified by entry.
    Return a nested list which can be fed to the data frame.
    
    Arguments: entry: the index of the dataframe. Not the id!
    '''
    ID = df_player_career_stats.loc[entry,"ID"];
    name = df_player_career_stats.loc[entry,"Name"];
    url = df_player_career_stats.loc[entry,"split_url"];

    response = requests.get(url, headers=headers);
    text = BeautifulSoup(response.text, 'html.parser');

    if response.status_code != 200:
        raise Exception(f"The status code is not 200! It is {response.status_code}.");

    pret = text.findAll("pre");

    for ta in pret:
        if ta.get_text().find("Total") != -1:
            clear_output();
            print("Found batting record chunk for " + name + '.');
            break;

    ltemp = ta.contents[0].splitlines();
    
    status = ltemp[0]; # The glossary line. Use this to detect what was missing.
    glossary = status.split();
    glossary_complete = (len(glossary) == 22);
    if not glossary_complete:
        glossary_index = [columns_career.index(x)-3 for x in glossary];
        # A mask, or indicator on where the glossary maps to in full list.
    
    player_career_split = [];
    for status in ltemp:
        # status_split = status.split();

        # Try to find true
        status_tag = [status.startswith(x) for x in situations];
        try:
            tag_num = status_tag.index(True);
        except:
            tag_num = -1;

        if glossary_complete:
            if tag_num in situations_no_g: # On tags where no G column is available, need to pad with a np.nan
                player_career_split.append([ID, name, situations[tag_num], np.nan] + \
                                          status[situations_len[tag_num]:].split());
            elif tag_num >= 0:
                player_career_split.append([ID, name, situations[tag_num]] + \
                                          status[situations_len[tag_num]:].split());
        else:
            if tag_num in situations_no_g:
                content = [np.nan] + status[situations_len[tag_num]:].split();
                content_dest = [np.nan] * (len(columns_career) - 3);
                for j in range(len(glossary)):
                    content_dest[glossary_index[j]] = content[j];
                player_career_split.append([ID, name, situations[tag_num]] + content_dest);
            elif tag_num >= 0:
                content = status[situations_len[tag_num]:].split();
                content_dest = [np.nan] * (len(columns_career) - 3);
                for j in range(len(glossary)):
                    content_dest[glossary_index[j]] = content[j];
                player_career_split.append([ID, name, situations[tag_num]] + content_dest);
    
    print([len(x) for x in player_career_split]);
    return player_career_split;


## Scraping the split pages and extend the dataframe.
Now this is where the scraping happens. Let's keep our fingers crossed and hope that we won't get blocked :)

In [182]:
df_player_career_split = pd.DataFrame(columns = columns_career);

In [231]:
for i in range(df_player_career_stats.shape[0]): # Ideally we want to do this, but need to debug once in a while.
# for i in range(15048,16009): # Use this to pick up where we are left over
    psp = scrape_player_split(i);
    df_player_career_split = df_player_career_split.append(pd.DataFrame(psp[0:], columns = columns_career));

Found batting record chunk for Dutch Zwilling.
[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]


Now the files are scraped, need some tiny processing and save the csv file...

In [269]:
for i in range(4,22):
    df_player_career_split[columns_career[i]] = df_player_career_split[columns_career[i]].str.replace('i','').astype(float); #apply(lambda x: int(x) if isinstance(x, str) else x)

for i in range(22,25):
    df_player_career_split[columns_career[i]] = df_player_career_split[columns_career[i]].str.replace('-','');

In [270]:
df_player_career_split

,ID,Name,Situation,G,AB,R,H,2B,3B,HR,...,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG
0,0,David Aardsma,Total,331,4.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,.000,.000
1,0,David Aardsma,Home,181,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,.000,.000
2,0,David Aardsma,Away,150,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,.000,.000
3,0,David Aardsma,vs RHP,NaN,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,.000,.000
4,0,David Aardsma,vs LHP,NaN,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,.000,.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413966,19917,Dutch Zwilling,AT CF,335,1245.0,164.0,359.0,74.0,15.0,30.0,...,28.0,0.0,0.0,14.0,2.0,46.0,21.0,.288,.356,.444
413967,19917,Dutch Zwilling,AT RF,5,8.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,.125,.125,.125
413968,19917,Dutch Zwilling,AT PH,27,25.0,1.0,4.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,.160,.192,.200
413969,19917,Dutch Zwilling,AT PR,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,


In [272]:
df_player_career_split.to_csv("player_career_split_data.csv")
print(df_player_career_split.shape) # (413971, 25)

(413971, 25)


## The outcome at this stage

Get a pandas data frame, called df_play_career_split, which looked like below:

In [258]:
df_player_career_split

,ID,Name,Situation,G,AB,R,H,2B,3B,HR,...,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG
0,0,David Aardsma,Total,331,4,0,0,0,0,0,...,1,0,0,0,0,0,0,.000,.000,.000
1,0,David Aardsma,Home,181,2,0,0,0,0,0,...,0,0,0,0,0,0,0,.000,.000,.000
2,0,David Aardsma,Away,150,2,0,0,0,0,0,...,1,0,0,0,0,0,0,.000,.000,.000
3,0,David Aardsma,vs RHP,NaN,2,0,0,0,0,0,...,1,0,0,0,0,0,0,.000,.000,.000
4,0,David Aardsma,vs LHP,NaN,2,0,0,0,0,0,...,0,0,0,0,0,0,0,.000,.000,.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,19917,Dutch Zwilling,AT CF,335,1245,164,359,74,15,30,...,28,0,0,14,2,46,21,.288,.356,.444
29,19917,Dutch Zwilling,AT RF,5,8,1,1,0,0,0,...,1,0,0,0,0,0,0,.125,.125,.125
30,19917,Dutch Zwilling,AT PH,27,25,1,4,1,0,0,...,1,0,0,1,0,0,0,.160,.192,.200
31,19917,Dutch Zwilling,AT PR,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-,-,-


And the data frame is saved to the file `player_career_split_data.csv`. Up to this point, we retrieve the data for all but 7 players who batted in the MLB, which resulted in 16002 player records.

## ***Dangerous. Don't touch***

In [268]:
# If messed up with the variables, do this...
# columns_dtype = [str]*25;
# columns_dtype_dict = dict(zip(columns_career, columns_dtype));

# df_player_career_split = pd.read_csv("player_career_split_data.csv", usecols = columns_career, dtype = columns_dtype_dict);